In [1]:
import sys
!{sys.executable} -m pip install python-binance nltk psaw pandas praw

In [2]:
subreddit = "AltStreetBets"

In [3]:
from psaw import PushshiftAPI

api = PushshiftAPI()


In [4]:
import datetime as dt

delta =  dt.timedelta(days=7)
start_epoch = int((dt.datetime.now() - delta).timestamp())

start_epoch

1612717174

In [5]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
def get_sentiment(value, version):
    return sia.polarity_scores(value)[version]


In [7]:
# posts = list(api.search_submissions(after=start_epoch,
#                             subreddit=subreddit,
#                             filter=['title','selftext', 'link_flair_text'],
#                             limit=1000))


import praw,psaw
import pandas as pd

with open('reddit_keys.txt') as f:
    api_keys = f.read().splitlines()

reddit = praw.Reddit(
    client_id=api_keys[0],
    client_secret=api_keys[1],
    user_agent="NLP-Comments"
)

api = psaw.PushshiftAPI(r=reddit)

submissions = reddit.subreddit(subreddit).top(limit=10)

dicts = []

for submission in submissions:
    submission.comments.replace_more(limit=None)
    
    df_dict = {
        "title": submission.title,
        "selftext": submission.selftext,
        "score": submission.score,
        "upvote_ratio": submission.upvote_ratio
    }
    
    comments_sentiments = []
    for comment in submission.comments.list():
        comments_sentiments.append({
            "pos": get_sentiment(comment.body, "pos"),
            "neu": get_sentiment(comment.body, "neu"),
            "neg": get_sentiment(comment.body, "neg")
        })
    comments_sentiments_df = pd.DataFrame(comments_sentiments)
    
    df_dict["comments_pos_avg"] = comments_sentiments_df.pos.mean()
    df_dict["comments_neu_avg"] = comments_sentiments_df.neu.mean()
    df_dict["comments_neg_avg"] = comments_sentiments_df.neg.mean()
    dicts.append(df_dict)
    

df = pd.DataFrame(dicts)
df


,title,selftext,score,upvote_ratio,comments_pos_avg,comments_neu_avg,comments_neg_avg
0,BAN all the Dogecoins and pump and dumps shillers,We need to ban all the pump and dumps. This is...,855,0.92,0.110478,0.797237,0.092290
1,Doge .0875 buyers be like.,,637,0.98,0.129429,0.775714,0.094846
2,The unfortunate truth,,642,0.98,0.129691,0.788055,0.082182
3,This subreddit in a nutshell,,608,0.98,0.156000,0.754520,0.089440
4,Thank god for alts!,,584,0.96,0.136318,0.799147,0.064558
5,"If you buy doge or promote it, you're fucking ...",Were here to make money and to push the crypto...,563,0.87,0.158351,0.731351,0.105400
6,Our path to glory........,,494,0.97,0.163143,0.770531,0.066327
7,One of these is not like the others,,489,0.96,0.151623,0.777679,0.070698
8,When Uber starts accepting btc,,479,0.96,0.146103,0.791486,0.062411
9,🚀🚀🚀NANO GAIN PORN🚀🚀🚀,,479,0.95,0.168426,0.715778,0.097333


In [8]:
df['title_positive'] = df.title.apply(get_sentiment, version='pos')
df['title_neutral'] = df.title.apply(get_sentiment, version='neu')
df['title_negative'] = df.title.apply(get_sentiment, version='neg')

df['text_positive'] = df.selftext.apply(get_sentiment, version='pos')
df['text_neutral'] = df.selftext.apply(get_sentiment, version='neu')
df['text_negative'] = df.selftext.apply(get_sentiment, version='neg')

df['text_positive'] = df.selftext.apply(get_sentiment, version='pos')
df['text_neutral'] = df.selftext.apply(get_sentiment, version='neu')
df['text_negative'] = df.selftext.apply(get_sentiment, version='neg')


col_pos = df[["title_positive","text_positive","comments_pos_avg"]]
col_neg = df[["title_negative","text_negative","comments_neg_avg"]]
col_neu = df[["title_neutral","text_neutral","comments_neu_avg"]]


# col_pos

df['positive'] = col_pos.mean(axis=1)
df['negative'] = col_neg.mean(axis=1)
df['neutral'] = col_neu.mean(axis=1)

df['invest_now'] = ( df['positive'] > df['negative']) & (df ['positive'] > df['neutral'] )

df

,title,selftext,score,upvote_ratio,comments_pos_avg,comments_neu_avg,comments_neg_avg,title_positive,title_neutral,title_negative,text_positive,text_neutral,text_negative,positive,negative,neutral,invest_now
0,BAN all the Dogecoins and pump and dumps shillers,We need to ban all the pump and dumps. This is...,855,0.92,0.110478,0.797237,0.092290,0.000,0.499,0.501,0.000,0.774,0.226,0.036826,0.273097,0.690079,False
1,Doge .0875 buyers be like.,,637,0.98,0.129429,0.775714,0.094846,0.385,0.615,0.000,0.000,0.000,0.000,0.171476,0.031615,0.463571,False
2,The unfortunate truth,,642,0.98,0.129691,0.788055,0.082182,0.365,0.159,0.476,0.000,0.000,0.000,0.164897,0.186061,0.315685,False
3,This subreddit in a nutshell,,608,0.98,0.156000,0.754520,0.089440,0.000,1.000,0.000,0.000,0.000,0.000,0.052000,0.029813,0.584840,False
4,Thank god for alts!,,584,0.96,0.136318,0.799147,0.064558,0.710,0.290,0.000,0.000,0.000,0.000,0.282106,0.021519,0.363049,False
5,"If you buy doge or promote it, you're fucking ...",Were here to make money and to push the crypto...,563,0.87,0.158351,0.731351,0.105400,0.161,0.556,0.283,0.184,0.672,0.144,0.167784,0.177467,0.653117,False
6,Our path to glory........,,494,0.97,0.163143,0.770531,0.066327,0.000,1.000,0.000,0.000,0.000,0.000,0.054381,0.022109,0.590177,False
7,One of these is not like the others,,489,0.96,0.151623,0.777679,0.070698,0.000,0.768,0.232,0.000,0.000,0.000,0.050541,0.100899,0.515226,False
8,When Uber starts accepting btc,,479,0.96,0.146103,0.791486,0.062411,0.418,0.582,0.000,0.000,0.000,0.000,0.188034,0.020804,0.457829,False
9,🚀🚀🚀NANO GAIN PORN🚀🚀🚀,,479,0.95,0.168426,0.715778,0.097333,0.630,0.370,0.000,0.000,0.000,0.000,0.266142,0.032444,0.361926,False
